# Python assignment


|Name|SNR|ANR|
|----|---|----|
|Elena Tello Peña|2077934|u224519|
|Guillem Torres Antillach|2071298|u518906|


## Research question

How does economic development relate to water withdrawals? The possible answer to this question is that of an Environmental Kuznets Curve (EKC), we will try to elucidate whether this is true through the analysis of various datasets (both cross-sectional and panel data) and different approaches for performing them.

## Motivation

We are now facing a global context of water resources crisis and its rates of consumption have even outpaced that of population growth. To properly tackle this issue, it is then necessary to estimate how water availability will be shapen in the future. In this sense, the analysis of data that is until now available sometimes shows an "inverted U" type relationship between economic growth and water withdrawals, which corresponds to the environmental equivalent of the Kuznets Curve (EKC). This shape would mean that when a country is in an early stage of economic development it will keep increasing its water use as long as it grows to the turning point (due to a higher deploy of industrial activities, higher consume, and other circumstances). When this point, the peak of the curve, is reached, the increase of economic development would help the country reduce this withdrawals, likely through technological advancements.

Our purpose is then to study how Gross National Product per capita, as a proxy for economic development, influences water withdrawals and also to find whether the cited EKC relationship is actually found when studying the data, trying to prove that there might be a way to predict how economic development can be determinant for the reshaping of the water consumption patterns.

## Data and results

We first conduct a general statistical analysis of the available World Bank Data related to water withdrawals, including a function showing GDPpc and water withdrawals per capita for any country that is selected in the input, bar graphs to show the differences of water use per sector and several maps that depict the water withdrawals per country. 

In order to assess the probable existance of an EKC, we first perform GLS regressions on a cross-sectional dataset and then fixed and random effects regressions on panel data. For both cases there are several assumptions that must be taken into account. The common one is the hypothesis of homoskedasticity in the error term that is tested through White and Breusch-Pagan tests: if the hypothesis is rejected then there is evidence of heteroskedasticity, as it is concluded when performing every one of the models. For the panel data regressions, we also look for serial autocorrelation through the Durbin-Watson test and we find evidence of it in every case. Also, for the models that include a quadratic term, we estimate the turning point of the function.

When implementing these regressions, we find that only the panel data ones show some degree of significance, especially the linear version of them. When adding the square term so as to prove the cited EKC relationship, only the agricultural sector water withdrawals show relevant results. 

This way, we cannot really conclude that the relationships that we study properly follow an EKC shape, but the reduced significance may be caused by other limitations as we will further detail in the conclusion section.
In both apporaches the sensitivity of the analysis is tested through implementation for the same tools on different data. 

### Overview of the data

In [ ]:
import geopandas as gpd
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
import sys
!{sys.executable} -m pip install mapclassify
import sys
!{sys.executable} -m pip install xlrd
from shapely.geometry import Point
import sys
!{sys.executable} -m pip install folium
import folium
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import het_breuschpagan
import math
!{sys.executable} -m pip install folium
import xlrd

## Example of Kuznets Curve
Below we can see the case of Spain (if you use the function you can get any country that you want). In this case we see how water per capita withdrawals decrease as gdp per capita increases.

In [ ]:
# The first two lines extract the data from the csv file and then merge it together using the inner join method. This method will return a dataframe that contains only the rows that share common characteristics 
gdppc = pd.read_csv("gdp-per-capita-worldbank.csv")
waterppc = pd.read_csv("water-withdrawals-per-capita.csv")
waterxgdp = gdppc.merge(waterppc, how = "inner")
# this line will set the column Year as the index of the dataframe
waterxgdp = waterxgdp.set_index('Year')
waterxgdp
fig, ax = plt.subplots(figsize=(18,5))
# what this function does basically:
# it takes a country of the dataframe, the one that the author wants to study (in this case India). then creates a time plot with two y axis, so you can compare the evolution of the two variables across time  
def country(data):
    ax.plot(data.index, data["GDP per capita, PPP (constant 2017 international $)"], color = "black", label = "GDP per capita")
    ax.set_xlabel("Time")
    ax.set_ylabel("GDP per capita, PPP (constant 2017 international $)")
    ax2 = ax.twinx()
    ax2.plot(data.index, data["Total water withdrawal per capita"], label = "water per capita")
    ax2.set_ylabel("Total water withdrawal per capita")
    ax.legend()
    ax2.legend(loc = 'upper right')
    plt.title("Comparison of Total water withdrawal per capita and GDP per capita, PPP (constant 2017 international $) by country: ")
    plt.show()
# The reason behind this complicated way to retrieve the country, is that the countries are part of an specific column named Entity
country(waterxgdp.loc[(waterxgdp["Entity"]=="Spain")])
print(waterxgdp.loc[(waterxgdp["Entity"]=="Spain")])

# Answers
For the GLS regression we obtain the following turning points (in annual GDP per capita terms): <br /> 
Total water withdrawal per capita: **10.51 dollars** <br /> 
Industry water withdrawal: **157 dollars** <br /> 
Agriculture water withdrawal: **32 dollars** <br /> 
Domestic water withdrawal: **17 dollars** <br /> 

# Data:
## Stacked Bar Plots:
In the following bar plot what we can see is that the weight of agriculture has gone down in the recent years while the weight of Industrial and Domestic has increased over the years. What this could mean is that more and more countries are starting to develop and they are leaving the low income area. 

In [ ]:
# reads the data from excel and transforms it into a dataframe
waterAgr = pd.read_excel("AgriculturalWater.xls", engine = "xlrd", header = 3 )
# drops rows with na values
waterAgr = waterAgr.dropna(0)
# calculates the mean of every row
avgwater = pd.DataFrame(waterAgr.mean(axis=0))
waterInd = pd.read_excel("IndustrialWater.xls", engine = "xlrd", header = 3 )
waterInd = waterInd.dropna(0)
avgwater["Industry"] = pd.DataFrame(waterInd.mean(axis=0))
waterDom = pd.read_excel("domestic_water.xls", engine = "xlrd", header = 3 )
waterDom = waterDom.dropna(0)
avgwater["Domestic"] = pd.DataFrame(waterDom.mean(axis=0))
# this line will store the first three columns of the database
avgwater = avgwater.iloc[: , :3]
# This chunk  of code creates the plot seen below.
fig, ax = plt.subplots(figsize=(18,5))
ax.bar(avgwater.index, avgwater[0], label = "Agriculture")
ax.bar(avgwater.index, avgwater["Domestic"], bottom=avgwater[0], label = "Domestic")
ax.bar(avgwater.index, avgwater["Industry"], bottom=avgwater[0]+avgwater["Domestic"], label = "Industry")
ax.set_xticklabels(avgwater.index, rotation = 90)
ax.set_ylabel("Percentage of total water withdrawals used by each sector")
ax.legend()
plt.show()


## Maps

In [ ]:

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#what this line of code does is the following: alphabethically sorts the dataframe world based on the unicode "NFKD" format and then resets the index.
#we call iloc because the code inside the [] will return a series of integers that need to be reordered
world1 = (world.iloc[world['name'].str.normalize('NFKD').argsort()]).reset_index()
#this line of code will drop Antartica, in the previous line what we did is to order alphabetically but also reorder the index
world1.drop(4, inplace = True)
water1 = pd.read_excel("TotalWaterMap.xls", engine = "xlrd", header = 3 )
# What this line of code does is obtain a list of countries that do not have an iso_a3 equal to the code of the excel database and then print the name of that country and what they have as an ISO code 
world1[~world1['iso_a3'].isin(list(water1['Country Code']))][['iso_a3', 'name']]
#This line updates the Iso Codes of the countries that do not have a correct ISO code
world1.loc[world1.name == 'France', 'iso_a3'] = 'FRA'
world1.loc[world1.name == 'Norway', 'iso_a3'] = 'NOR'
# in this line what we do is merge the two dataframes, we use how = left to maintain every row of the left dataframe which is the map in this case. 
water1 = world1.merge(water1,
                            left_on = 'iso_a3',
                            right_on = 'Country Code',
                            how = 'left',)
# what this function does: It takes a specific year of the dataframe water1, and also the dataframe itself and returns a colorised map that ranks the counntries by water withdrawals in cubic meters 
def newmap (date, water):
    #water per capita in billion cubic meters
    water["percapita"] = date/(water["pop_est"]/1000000000)
    # what this two lines do is create a map where the color of each country is determined by the percapita value
    axes = water.plot(column= "percapita",  edgecolor = "black", legend = True, legend_kwds={'shrink': 0.3}, cmap = "OrRd")
    plt.rcParams["figure.figsize"]=20,20
    axes.set_axis_off()
    plt.title("Annual freshwater withdrawals per capita cubic meters")
# this line sets the figure size of the map
    plt.show()
    
    
newmap(water1["2017"],water1)



In [ ]:
waterorder=water1.sort_values(by=['percapita'], ascending = False)
waterorder.head()


In [ ]:
waterInd = pd.read_excel("IndustrialWater.xls", engine = "xlrd", header = 3 )
waterDom = pd.read_excel("domestic_water.xls", engine = "xlrd", header = 3 )
waterAgr = pd.read_excel("AgriculturalWater.xls", engine = "xlrd", header = 3 )
# what this function does: it takes any year and the database that the author wants to collect information from, then it is merged with the map of the world of geopandas 
# what is singular of this map is the use of the if statement to change the title that is returned, we do this because we want to get information from three databases and we want to get a correct title for each of them. 
def newsectormap (date, water):
    water1 = world1.merge(water,
                            left_on = 'iso_a3',
                            right_on = 'Country Code',
                            how = 'left',)
    axes = water1.plot(column= date,  edgecolor = "black", legend = True, legend_kwds={'shrink': 0.3}, cmap = "OrRd")
    axes.set_axis_off()
    if water is waterInd:
        return plt.title("Annual freshwater withdrawals, Industry (% of total freshwater withdrawal " + date)
    if water is waterAgr:
        return plt.title("Annual freshwater withdrawals, Agriculture (% of total freshwater withdrawal " + date)
    if water is waterDom: 
        return plt.title("Annual freshwater withdrawals, Domestic (% of total freshwater withdrawal " + date)
    return none
    plt.rcParams["figure.figsize"]=20,20
    plt.show()

newsectormap("2012", waterInd)
newsectormap("2012", waterAgr)
newsectormap("2012", waterDom)
# this line sorts the value of the Agricultural water dataframe and gives us the top 5 countries that have a higher percentage of their use in this sector



### Overview of the Sectors:
In the year 2017 (we take this year because we do the regressions based on this specific year). We see big differences between the uses that country make of their water. Starting with the Industry one, we see countries using up to 96 of their total water withdrawals in this sector. Also in this sector we see more mix between developed countries (Estonia, Netherlands, Germany) and countries that are developing (Jamaica). On the other hand, we have the Agriculture sector, here we find that the top 5 share a specific characteristic: They are extremely underdeveloped. Finally we have the domestic sector, here the top 5 has also similar characteristics, all of them are small countries with no important industry and there economy is focused on the service sector (Tourism and Banking mostly).

In [ ]:
print(waterInd.sort_values(by=['2017'], ascending = False).head())
print(waterAgr.sort_values(by=['2017'], ascending = False).head())
print(waterDom.sort_values(by=['2017'], ascending = False).head())

### GLS Regression Analysis
We will use the following data:
Total water withdrawals per capita, Annual freshwater withdrawals Industry (% of total freshwater withdrawal), Annual freshwater withdrawals Agriculture (% of total freshwater withdrawals), Annual freshwater withdrawals, Domestic (% of total freshwater withdrawal and GDP per capita, PPP (constant 2017 international dollars). The reason that we use this databases is to get a better picture of how countries differ in their water uses, and also try to explain the different turning points that we get in the results. As an example: when a country starts to develop, it increases it's use of industrial water and domestic while the Agricultural one diminishes. <br />
The total withdrawals and GDP are in per capita terms while the Industry, Agriculture and Domestic are in percentage terms, the reason behing this is that Industry, Agriculture and Domestic were not available. <br />
To try an answer the question we will two econometric approaches: <br />
The first one is a cross sectional GLS. We use a reduced form with only one explanatory variable in this case GDP (in both linear and squared versions), which means we can only test for correlations not causation, we do this following the advide of the existing literature: \begin{equation} Water_i = \beta_0 + \beta_1GDP_i + \beta_2GDP_i + \epsilon_i \end{equation} <br />
The form of the regression is log log, we do it this way to correct for possible outliers in the data.
To perform a correct analysis what we do first is test for heteroscedasticity, using the breusch pagan test.
As we will see every sector rejects the null hypothesis meaning that we have heteroscedasticity.
After we have perform the GLS regression and obtained the coefficients we will then proceed and calculate the turning points of each sector.   
To calculate the turning use we will use the formula proposed by Song Tao(2008): \begin{equation} T=
e^{-\beta_1/(2*\beta_2)} \end{equation}

### Assumptions for the model:
We have three main assumptions for the GLS regression: <br />
**1st**: We perform a GLS to account for the Heteroscedasticity that is found in the data. waterInd.sort_values <br />
**2nd**:For the explanatory variables (GDP), we take the mean value of a period instead of taking a single year (as we do with water), we do this because we want to evaluate the turning point of the economy. <br />
**3rd**: The year that we choose for total water withdrawals changes from the one we took in the sector, the reason why is the unavailability of the data for that year. 


In [ ]:
gdppc.rename(columns={'Country Name': 'Entity'}, inplace=True)
gdppc = gdppc.pivot(index='Entity', columns='Year', values='GDP per capita, PPP (constant 2017 international $)')
waterppc = waterppc.pivot(index='Entity', columns='Year', values='Total water withdrawal per capita')
waterppc = waterppc.reset_index()
gdppc = gdppc.reset_index()

In [ ]:
gdppc.rename(columns={'Country Name': 'Entity'}, inplace=True)
gdppc['mean'] = gdppc.mean(axis=1)
gdppc["lGDPxc"] = np.log10(gdppc["mean"])
gdppc["lGDPxcsq"] = np.log10(gdppc["mean"])**2
waterpxc = waterppc.merge(gdppc, on = "Entity", how = "left" )
waterpxc = waterpxc.fillna(1)


def econometrics_water(water):
    waterpxc["newwater"] = np.log10(water)
    X = pd.DataFrame({"GDP": np.log10(waterpxc["lGDPxc"]), "GDP2": waterpxc["lGDPxcsq"]})
    f ='newwater~X'
    breusch_model = ols(formula=f, data=waterpxc).fit()
    bp_test = het_breuschpagan(breusch_model.resid, X)
    labels = ["Total Water Withdrawals LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test, p-value"]
    print( labels, bp_test)
    X = pd.DataFrame({"GDP": waterpxc["lGDPxc"], "GDP2": waterpxc["lGDPxcsq"]})
    X = sm.add_constant(X)
    Y = waterpxc["newwater"]
    GLS_MU_sigma = sm.GLS(Y,X)
    Fit_mu = GLS_MU_sigma.fit()
    coefficient = Fit_mu.params
    turning_point = np.exp(-coefficient[1]/(2*coefficient[2]))
    print(Fit_mu.summary(), turning_point)
    

econometrics_water(waterpxc["2015_x"])
waterppc
 

In [ ]:
gdpexcel = pd.read_excel("GDP2011_18.xls", engine = "xlrd", header = 3 )
water_sector = waterInd.merge(gdpexcel, on = "Country Name", how = "left")
water_sector = water_sector.dropna(0)
def econometrics_Ind(water):
    water_sector["newwater"] = np.log10(water)
    water_sector["lGDPxc"] = np.log10(water_sector["Average"])
    water_sector["lGDPxcsq"] = water_sector["lGDPxc"]**2
    X = pd.DataFrame({"GDP": water_sector["lGDPxc"], "GDP2": water_sector["lGDPxcsq"]})
    f ='newwater~X'
    breusch_model = ols(formula=f, data=water_sector).fit()
    bp_test = het_breuschpagan(breusch_model.resid, X)
    labels = ["Total Water Withdrawals LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test, p-value"]
    print( labels, bp_test)
    X = pd.DataFrame({"GDP": water_sector["lGDPxc"], "GDP2": water_sector["lGDPxcsq"]})
    X = sm.add_constant(X)
    Y = water_sector["newwater"]
    GLS_MU_sigma = sm.GLS(Y,X)
    Fit_mu = GLS_MU_sigma.fit()
    coefficient = Fit_mu.params
    turning_point = np.exp(-coefficient[1]/(2*coefficient[2]))
    print(Fit_mu.summary(), turning_point)
    
econometrics_Ind(water_sector["2017"])


In [ ]:
water_sector = waterAgr.merge(gdpexcel, on = "Country Name", how = "left")
water_sector = water_sector.dropna(0)
def econometrics_Agr(water):
    water_sector["newwater"] = np.log10(water)
    water_sector["lGDPxc"] = np.log10(water_sector["Average"])
    water_sector["lGDPxcsq"] = water_sector["lGDPxc"]**2
    X = pd.DataFrame({"GDP": water_sector["lGDPxc"], "GDP2": water_sector["lGDPxcsq"]})
    f ='newwater~X'
    breusch_model = ols(formula=f, data=water_sector).fit()
    bp_test = het_breuschpagan(breusch_model.resid, X)
    labels = ["Total Water Withdrawals LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test, p-value"]
    print( labels, bp_test)
    X = pd.DataFrame({"GDP": water_sector["lGDPxc"], "GDP2": water_sector["lGDPxcsq"]})
    X = sm.add_constant(X)
    Y = water_sector["newwater"]
    GLS_MU_sigma = sm.GLS(Y,X)
    Fit_mu = GLS_MU_sigma.fit()
    coefficient = Fit_mu.params
    turning_point = np.exp(-coefficient[1]/(2*coefficient[2]))
    print(Fit_mu.summary(), turning_point)
    
econometrics_Agr(water_sector["2017"])


In [ ]:
gdppc=gdppc.rename(columns={"Entity": "Country Name"})
waterpxc1 = waterDom.merge(gdppc, on = "Country Name", how = "inner" )
waterpxc1 = waterpxc.fillna(1)

In [ ]:



def econometrics_Dom(water):
    waterpxc1["newwater"] = np.log10(water)
    X = pd.DataFrame({"GDP": np.log10(waterpxc1["lGDPxc"]), "GDP2": waterpxc1["lGDPxcsq"]})
    f ='newwater~X'
    breusch_model = ols(formula=f, data=waterpxc1).fit()
    bp_test = het_breuschpagan(breusch_model.resid, X)
    labels = ["Total Water Withdrawals LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test, p-value"]
    print( labels, bp_test)
    X = pd.DataFrame({"GDP": waterpxc1["lGDPxc"], "GDP2": waterpxc1["lGDPxcsq"]})
    X = sm.add_constant(X)
    Y = waterpxc1["newwater"]
    GLS_MU_sigma = sm.GLS(Y,X)
    Fit_mu = GLS_MU_sigma.fit()
    coefficient = Fit_mu.params
    turning_point = np.exp(-coefficient[1]/(2*coefficient[2]))
    print(Fit_mu.summary(), turning_point)

econometrics_Dom(waterpxc1[2017]) 
gdppc

### Turning point calculations
For the GLS regression we obtain the following turning points (in annual GDP per capita terms): <br /> 
Total water withdrawal per capita: **10.51 dollars** <br /> 
Industry water withdrawal: **157 dollars** <br /> 
Agriculture water withdrawal: **32 dollars** <br /> 
Domestic water withdrawal: **1736 dollars** <br /> 

# Sensitivity Analysis:
Instead of taking 2017 and 2015, we take 2012 and 2010 respectively. <br />
Total Water Witdrawals per capita:

In [ ]:
econometrics_water(waterpxc["2010_x"])

Industrial Water Witdrawals:

In [ ]:
econometrics_Ind(water_sector["2012_x"])

Agricultural Water Witdrawals:

In [ ]:
econometrics_Agr(water_sector["2012_x"])

Domestic Water Witdrawals:

In [ ]:
econometrics_Dom(waterpxc1[2012]) 

### Turning point calculations for the sensitivity analysis
For the GLS regression we obtain the following turning points (in annual GDP per capita terms): <br /> 
Total water withdrawal per capita: **6 dollars** <br /> 
Industry water withdrawal: **25 dollars** <br /> 
Agriculture water withdrawal: **28 dollars** <br /> 
Domestic water withdrawal: **10832 dollars** <br /> 

## PANEL DATA ANALYSIS

### INTRODUCTION

Now we proceed to carry out a panel data analysis. The purpose of this section is to further explore the EKC hypothesis through more accurate channels. Thanks to panel data we are able to reflect trends overtime, which is after all a key element of the EKC characteristics. Also, it helps to capture the variations that are likely to happen among nations due to geographical and climatic factors that may be behind differences in water use. 
We choose a more reduced database than in the previous step because panel data is not as widely available as cross-sectional data. This way, we choose 32 OECD countries to perform the study, also dividing water use by sectors: agricultural, domestic, and industrial. Although fixed effects regressions are better suited for the characteristics of these data, random effects are also taken into consideration so as to compare both regression approaches. The reason why fixed effects should be more convenient despite the final outcome in terms of significance, R-squared values or any other measure of statistical accuracy is due to the already mentioned fact that the entity intercept included in the fixed effects regressions allows for differences between water resource endowments that are likely to have an influence on water withdrawals, as well as climatic or state-specific factors that would determine the final water use. The equations for the panel fixed effect regressions would be as follows:

$$WATER_{i}= \beta_{0} + \alpha_{i} + \beta_{1}GDPpc_{it} + \epsilon_{it}$$
$$WATER_{i}= \beta_{0} + \alpha_{i} + \beta_{1}GDPpc_{it} + \beta_{2}{GDPpc^{2}_{it}}+ \epsilon_{it}$$

The variables WATER and GDPpc represent the logs of per capita annual water withdrawals in cubic meters and per capita gross domestic product in constant 2015 US$.


### CREATING THE DATASET: DATA ANALYSIS

In [ ]:
#install packages
!pip install wbdata
!pip install linearmodels
#import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import wbdata as wb
import seaborn as sns
import numpy.linalg as la
from scipy import stats
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from statsmodels.stats.diagnostic import het_white, het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
import statsmodels.api as sm

We first create a subset with all 38 OECD countries to be analysed and with the years for which freshwater withdrawals are available. 

In [ ]:
#Subsets of countries and years to be analyzed
OECDcountries = ['AUS','AUT','BEL','CAN','CHL', 'COL','CRI','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','ISL','IRL','ISR','ITA','JPN','KOR','LVA','LTU','LUX','MEX','NLD','NZL','NOR','POL','PRT','SVK','SVN','ESP','SWE','CHE','TUR','GBR','USA']
selyears = ['1982','1987','1992','1997','2002','2007','2012','2007','2012','2017']

Making use of the world bank database, we will look for the necessary variables, which are named this way:

-“wtrwth”: freshwater withdrawals, total (billion cubic meters)\
-“gdppc”: GDP per capita (constant 2015 US dollars)\
-“pop”: total population\
-“wtrwthag”: freshwater withdrawals, agriculture (percentage of total freshwater withdrawal)\
-“wtrwthdom”: freshwater withdrawals, domestic (percentage of total freshwater withdrawal)\
-“wtrwthind”: freshwater withdrawals, industrial (percentage of total freshwater withdrawal)\

The preliminary look for this database is thus displayed for the selected countries:

In [ ]:
#Dataframe building
fwindicators = {
                "ER.H2O.FWTL.K3" : "wtrwth",
                "NY.GDP.PCAP.KD" : "gdppc", #GDP per capita (constant 2015 US$)
                "SP.POP.TOTL" : "pop",
                "ER.H2O.FWAG.ZS" : "wtrwthag",
                "ER.H2O.FWDM.ZS" : "wtrwthdom",
                "ER.H2O.FWIN.ZS" : "wtrwthind"}
dfw = wb.get_dataframe(fwindicators, country=OECDcountries)

dfw.head()

Then, we convert the water withdrawals variables to per capita values by dividing for the population values. For the sake of this matter, in the case of the sectorial variables, since they come in percentage, we first multiply by the previously per capita values of the general freshwater withdrawals. 

In [ ]:
#Creation of per capita variables in cubic meters
dfw['wtrwthpc']=(dfw['wtrwth']/dfw['pop'])*1000000000 
#Transformation of percentage to cubic meters per capita in the next three lines of code
dfw['wtrwthagpc']=dfw['wtrwthag']*(dfw['wtrwth']/dfw['pop'])*10000000 
dfw['wtrwthdompc']=dfw['wtrwthdom']*(dfw['wtrwth']/dfw['pop'])*10000000 
dfw['wtrwthindpc']=dfw['wtrwthind']*(dfw['wtrwth']/dfw['pop'])*10000000
dfw.head()

The index is reset in order to better manipulate the data frame and the date values are then converted into numeric format, also selecting the subset of years that were previously stated. The preview shows how missing values are progressively decreasing and a summary of the statistics for each variable is shown. 

In [ ]:
#Resetting the index in order to manipulate the dataframe
dfw2=dfw.copy()
dfw2.reset_index(inplace=True)
dfw2.head()

In [ ]:
#Selecting the years subject of analysis
dfw2['date']=pd.to_numeric(dfw2['date'])
dfw_selyrs=dfw2[dfw2['date'].isin(pd.to_numeric(selyears))]
dfw_selyrs.head()

In [ ]:
#Dataframe summary statistics
dfw.describe()

Since it is more convenient to work with a balanced panel dataset, we look for missing values for both date and country index components. In the next step, we drop them out of the data frame.

In [ ]:
#Checking for missing values
dfnowater=dfw_selyrs[dfw_selyrs["wtrwth"].isnull()]
dfnowater['date'].value_counts()
dfnowater['country'].value_counts()
dfnogdp=dfw_selyrs[dfw_selyrs["gdppc"].isnull()]
dfnogdp['date'].value_counts()
dfnogdp['country'].value_counts()
dfnopop=dfw_selyrs[dfw_selyrs["pop"].isnull()]
dfnopop['date'].value_counts()
dfnopop['country'].value_counts()

In [ ]:
#Drop missing values and show balanced panel data
dfw_full=dfw_selyrs.dropna()
dfw_full.head()

We can now see the total number of observations and the type of each variable included in the data.

In [ ]:
#Showing characteristics of the balanced data set
dfw_full.info()

First of all, even if the dataset seems to be ready for the regression analysis, it is necessary to properly declare country and date as a multi-index component, otherwise the panel data analysis regression tool will not be able to identify the entity and time variable adequately. 

In [ ]:
#Set multiindex for the regressions
dfreg=dfw_full.copy()
dfreg.set_index(['country','date'], inplace=True)
dfreg.head()

##### Some graphs to further analyse the changes in the variables over time

We perform several plots in order to specifically see the differences in GDPpc and water withdrawals (total and by sector) across time of some countries of our choice (in this case, we picked a European country, an Asian country and an American country; the concrete choice was purely made out of convinience, since France, Korea and United States present data for the whole time span). 

In the first graph we can see that Korea shows the steeper GDPpc growth curve, while United States and France economics are closer to stagnation, which is coherent with reality.

In [ ]:
dfmani = dfreg.copy()
dfmani.reset_index(inplace=True)
dfmani = dfmani[dfmani['country'].isin(['France', 'United States', 'Korea, Rep.'])]
dfmani.sort_values(['date', 'country'], inplace=True)
dfmani.head()
plt.figure(figsize=(10, 8))
sns.lineplot(x='date', y='gdppc', hue='country', data=dfmani)
plt.yscale('log')
plt.legend(title='country', bbox_to_anchor=(1.05, 1), loc='upper left')

In the case of water withdrawals per capita we can see that the United States is far above the others and we can see how Korea goes above France after 1996 approximatedly, likely due to its economic development. 

In [ ]:
dfmani = dfreg.copy()
dfmani.reset_index(inplace=True)
dfmani = dfmani[dfmani['country'].isin(['France', 'United States', 'Korea, Rep.'])]
dfmani.sort_values(['date', 'country'], inplace=True)
dfmani.head()
plt.figure(figsize=(10, 8))
sns.lineplot(x='date', y='wtrwthpc', hue='country', data=dfmani)
plt.yscale('log')
plt.legend(title='country', bbox_to_anchor=(1.05, 1), loc='upper left')

In the next graph we see how United States and Korea are clearly more focused on agriculture than France is, given the levels of the functions.

In [ ]:
dfmani = dfreg.copy()
dfmani.reset_index(inplace=True)
dfmani = dfmani[dfmani['country'].isin(['France', 'United States', 'Korea, Rep.'])]
dfmani.sort_values(['date', 'country'], inplace=True)
dfmani.head()
plt.figure(figsize=(10, 8))
sns.lineplot(x='date', y='wtrwthagpc', hue='country', data=dfmani)
plt.yscale('log')
plt.legend(title='country', bbox_to_anchor=(1.05, 1), loc='upper left')

We now make a closer look on Korea's agricultural water withdrawals and see how this shape could resemble an EKC: the withdrawals maxed when economic development was growing at a higher rate and then started to decrease.

In [ ]:
dfmani = dfreg.copy()
dfmani.reset_index(inplace=True)
dfmani = dfmani[dfmani['country'].isin(['Korea, Rep.'])]
dfmani.sort_values(['date', 'country'], inplace=True)
dfmani.head()
plt.figure(figsize=(10, 8))
sns.lineplot(x='date', y='wtrwthagpc', hue='country', data=dfmani)
plt.yscale('log')
plt.legend(title='country', bbox_to_anchor=(1.05, 1), loc='upper left')

For the domestic withdrawals the difference between countries shrinks and we still see United States as the leader, also it is clear how France is gradually diminishing its withdrawals and Korea too.

In [ ]:
dfmani = dfreg.copy()
dfmani.reset_index(inplace=True)
dfmani = dfmani[dfmani['country'].isin(['France', 'United States', 'Korea, Rep.'])]
dfmani.sort_values(['date', 'country'], inplace=True)
dfmani.head()
plt.figure(figsize=(10, 8))
sns.lineplot(x='date', y='wtrwthdompc', hue='country', data=dfmani)
plt.yscale('log')
plt.legend(title='country', bbox_to_anchor=(1.05, 1), loc='upper left')

Finally, we can check the industrial sector and see a very different pattern. As we said before, it is clear that Korea is more focused in agricultural than in industrial activites in relative terms to France, which is now above the Asian country in water withdrawals per capita. Anyway, United States reaches once again the highest values.

In [ ]:
dfmani = dfreg.copy()
dfmani.reset_index(inplace=True)
dfmani = dfmani[dfmani['country'].isin(['France', 'United States', 'Korea, Rep.'])]
dfmani.sort_values(['date', 'country'], inplace=True)
dfmani.head()
plt.figure(figsize=(10, 8))
sns.lineplot(x='date', y='wtrwthindpc', hue='country', data=dfmani)
plt.yscale('log')
plt.legend(title='country', bbox_to_anchor=(1.05, 1), loc='upper left')

#### Final dataset fixing 

We now make sure that the time index has a numeric data type and the entity an object form.

In [ ]:
#Specifying the data types of the index components
date = dfreg.index.levels[1].astype(int)
country = dfreg.index.levels[0].astype(object)
dfreg.index = dfreg.index.set_levels([country, date])
dfreg.info()

For the regressions, we also need to include both the independent and dependent variables in their logarithm form, so as to linearize the variables. 

In [ ]:
#Adding the log terms to the regression dataframe
columns = ['wtrwthpc', 'gdppc', 'wtrwthagpc', 'wtrwthdompc', 'wtrwthindpc']
for col in columns:
    dfreg["log-" + col] = np.log(dfreg[col])
dfreg.head()

The same happens with the quadratic term that we need for the GDPpc variable in order to include this specification of the model.

In [ ]:
#Adding the squared term to the regression dataframe
for col in ['log-gdppc']:
    dfreg['sq_' + col] = np.square(dfreg[col])
    
dfreg.head()

## PANEL DATA REGRESSIONS

Next, we start with the regression analysis itself. Although we are going to do a panel regression, we start by executing a pooled OLS analysis to be able to test for homoskedasticity and serial autocorrelation, which are some of the assumptions related to a recommended use of fixed or random effects regressions rather than simple OLS regressions. In this sense, if we found no presence of heteroskedasticity or serial autocorrelation we would not be in the need of running a fixed or random effect regression. However, as we will see, none of these assumptions hold for a pooled OLS regression to be the ideal option, so we then perform the fixed and random effect regressions.

### LINEAR REGRESSIONS

We then start by declaring the independent ($GDPpc_{it}$) and dependent ($WATER_{it}$). We then perform the regression and store the residuals values for analysing whether there is or not homoskedasticity. We show a scatter plot of the residuals in order to see the way they are shaped. In this case, there is not a clear patter, which would make us contemplate heteroskedasticity as more likely. 

In [ ]:
#Setting the indepent and dependent variables
from linearmodels import PooledOLS
exog = sm.add_constant(dfreg['log-gdppc'])
endog = dfreg['log-wtrwthpc']

In [ ]:
#Pooled OLS for homoskedasticity 
mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)

In [ ]:
#Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

In [ ]:
#Homoskedasticity
#Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'green')
ax.axhline(0, color = 'b', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 10)
ax.set_ylabel('Residuals', fontsize = 10)
ax.set_title('Homoskedasticity Test', fontsize = 15)
plt.show()

However, for making sure that this is the case, we execute both the White and the Breusch-Pagan tests, for which the null hypotheses are both that there is evidence of homoskedasticity. In both cases the p-value is below 0.05, which indicates that the null hypothesis can be rejected and there is no evidence of homoskedasticity, and then there is evidence of heteroskedasticity in the errors terms. 

In [ ]:
#White-Test
pooled_OLS_dataset = pd.concat([dfreg, residuals_pooled_OLS], axis=1)
exog = sm.tools.tools.add_constant(dfreg['log-gdppc']).fillna(0)
white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

#Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

Subsequently, we look for serial correlation. For this matter we perform a Durbin-Watson test. The Durbin-Watson test will have one output between 0 – 4. The mean (= 2) would indicate that there is no autocorrelation identified, 0 – 2 means positive autocorrelation (the nearer to zero the higher the correlation), and 2 – 4 means negative autocorrelation (the nearer to four the higher the correlation). We then find out that the value shows a strong positive correlation. All in all, we must then privilege the application of fixed (or random) effects regressions rather than OLS ones. 

In [ ]:
#Non-Autocorrelation
# Durbin-Watson-Test
durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)

We then use the PanelOLS and RandomEffects specifications to do the analysis. The output shows that in both regressions the variables are strongly significant, and that the GDP per capita variable is negative. While the R-squared is higher for the random effects model, the log-likelihood shows to be better for the fixed effects. Anyway, both specifications show similar results and according to what was previously stated we rather stick to the fixed effects for theoretical coherence. 

In [ ]:
# FE und RE model
exog = sm.add_constant(dfreg['log-gdppc'])
endog = dfreg['log-wtrwthpc']
# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
#print results
print(re_res)
print(fe_res)

Finally, we perform a Hausman test in which we can conclude whether fixed or random effects are more convenient. The null hypothesis is that random effects are the preferred option and the alternative otherwise. In this case the p-value>0.05, so that we would not be able to reject the null hypothesis. However, as we have said already, it would make more sense to use a fixed-effect approach for theoretical coherence. This test shows that random effect would nonetheless be a good option for the present case. 

In [ ]:
#Hausman test
def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov
    v_B = re.cov
    df = b[np.abs(b) < 1e8].size
    chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B)) 
 
    pval = stats.chi2.sf(chi2, df)
    return chi2, df, pval
hausman_results = hausman(fe_res, re_res) 
print('chi-Squared: '  + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

#### SECTOR ANALYSIS: AGRICULTURAL

We then take into account the agricultural water withdrawals instead of the total ones. The procedure is the same as before.

In [ ]:
exog = sm.add_constant(dfreg['log-gdppc'])
endog = dfreg['log-wtrwthagpc']

In [ ]:
mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)

In [ ]:
# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

Again, the graph does not show a clear pattern, but the dots seem to be slightly more equally distributed than in the previous scenario. When we perform the tests, we see that indeed the p-value is not lower than 0.05 but since it can still be considered as significant (p-value<0.1) we cannot rule out the presence of heteroskedasticity.

In [ ]:
#Homoskedasticity
#Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'green')
ax.axhline(0, color = 'b', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 10)
ax.set_ylabel('Residuals', fontsize = 10)
ax.set_title('Homoskedasticity Test', fontsize = 15)
plt.show()

In [ ]:
#White-Test
pooled_OLS_dataset = pd.concat([dfreg, residuals_pooled_OLS], axis=1)
exog = sm.tools.tools.add_constant(dfreg['log-gdppc']).fillna(0)
white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

#Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

Serial autocorrelation is again present as the high correlation positive value shows.

In [ ]:
#Non-Autocorrelation
# Durbin-Watson-Test
durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)

In both the following regressions the coefficients are again strongly significant and negative, and we can see the same characteristics of the R-squared and log-likelihood as in the previous case. 

In [ ]:
# FE und RE model
exog = sm.add_constant(dfreg['log-gdppc'])
endog = dfreg['log-wtrwthagpc']
# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
#print results
print(re_res)
print(fe_res)

The Hausman test again indicates a preferred use of random effects, showing the importance of taking into account the difference between a purely statistic scope and a theory-based one, since the outcomes can vary a lot. 

In [ ]:
def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov
    v_B = re.cov
    df = b[np.abs(b) < 1e8].size
    chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B)) 
 
    pval = stats.chi2.sf(chi2, df)
    return chi2, df, pval
hausman_results = hausman(fe_res, re_res) 
print('chi-Squared:'  + str(hausman_results[0]))
print('degrees of freedom:' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

#### SECTOR ANALYSIS: DOMESTIC

Now we then take into account the domestic water withdrawals instead of the total ones. The procedure is once more the same as before.

In [ ]:
exog = sm.add_constant(dfreg['log-gdppc'])
endog = dfreg['log-wtrwthdompc']

In [ ]:
mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)

In [ ]:
# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

Again, the graph does not show a clear pattern although the dots are more concentrated. Both the tests lead us again to conclude that heteroskedasticity cannot be ruled out (p-value<0). 

In [ ]:
#Homoskedasticity
#Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'green')
ax.axhline(0, color = 'b', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 10)
ax.set_ylabel('Residuals', fontsize = 10)
ax.set_title('Homoskedasticity Test', fontsize = 15)
plt.show()

In [ ]:
#White-Test
pooled_OLS_dataset = pd.concat([dfreg, residuals_pooled_OLS], axis=1)
exog = sm.tools.tools.add_constant(dfreg['log-gdppc']).fillna(0)
white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

#Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

Serial autocorrelation is once again positive and strong.

In [ ]:
#Non-Autocorrelation
# Durbin-Watson-Test
durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)

For the regressions, we find a difference now, as only the fixed-effects specification shows a significant coefficient for GDP pc, and it is again negative. Logically, the Hausman test now shows a preference for fixed-effects (p-value<0.5).

In [ ]:
# FE und RE model
exog = sm.add_constant(dfreg['log-gdppc'])
endog = dfreg['log-wtrwthdompc']
# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
#print results
print(re_res)
print(fe_res)

In [ ]:
def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov
    v_B = re.cov
    df = b[np.abs(b) < 1e8].size
    chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B)) 
 
    pval = stats.chi2.sf(chi2, df)
    return chi2, df, pval
hausman_results = hausman(fe_res, re_res) 
print('chi-Squared:'  + str(hausman_results[0]))
print('degrees of freedom:' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

#### SECTOR ANALYSIS: INDUSTRIAL

Finally, we analyse the industrial water withdrawals.

In [ ]:
exog = sm.add_constant(dfreg['log-gdppc'])
endog = dfreg['log-wtrwthindpc']

In [ ]:
mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)

In [ ]:
# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

The scatter plot is again diffuse and both White and Breusch-Pagan tests show small p-values, which indicates that heteroskedasticity in the error terms cannot be discarded. 

In [ ]:
#Homoskedasticity
#Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'green')
ax.axhline(0, color = 'b', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 10)
ax.set_ylabel('Residuals', fontsize = 10)
ax.set_title('Homoskedasticity Test', fontsize = 15)
plt.show()

In [ ]:
#White-Test
pooled_OLS_dataset = pd.concat([dfreg, residuals_pooled_OLS], axis=1)
exog = sm.tools.tools.add_constant(dfreg['log-gdppc']).fillna(0)
white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

#Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

Autocorrelation is positive and strong.

In [ ]:
#Non-Autocorrelation
# Durbin-Watson-Test

durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)

The output shows highly significant coefficients for both specifications, but the Hausman test corroborates this time the use of fixed effects (p-value<0).

In [ ]:
# FE und RE model
exog = sm.add_constant(dfreg['log-gdppc'])
endog = dfreg['log-wtrwthindpc']
# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
#print results
print(re_res)
print(fe_res)

In [ ]:
def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov
    v_B = re.cov
    df = b[np.abs(b) < 1e8].size
    chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B)) 
 
    pval = stats.chi2.sf(chi2, df)
    return chi2, df, pval
hausman_results = hausman(fe_res, re_res) 
print('chi-Squared:'  + str(hausman_results[0]))
print('degrees of freedom:' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

 ### QUADRATIC MODELS

In order to further approach the inverted-U shape of the EKC, we now perform a quadratic variation of the previous regressions. All the steps are conducted the same way as before. 

In [ ]:
#Setting the indepent and dependent variables
exog_vars = ['log-gdppc','sq_log-gdppc']
exog = sm.add_constant(dfreg[exog_vars])
endog = dfreg['log-wtrwthpc']

In [ ]:
#Pooled OLS for homoskedasticity 
mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)

We include a graph that shows that the data follows a parabolic trend using the seaborn tool. 

In [ ]:
sns.set_theme(color_codes=True)
sns.regplot(x="log-gdppc", y="log-wtrwthpc", data=dfreg, order=2)

In [ ]:
#Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

The homoskedasticity graph is not concluding and the White and Breusch-Pagan tests show different p-values, the first one indicating the presence of heteroskedasticity and the second one not doing so. Facing this ambiguity, it is not possible to totally discard heteroskedasticity. 

In [ ]:
#Homoskedasticity
#Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'green')
ax.axhline(0, color = 'b', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 10)
ax.set_ylabel('Residuals', fontsize = 10)
ax.set_title('Homoskedasticity Test', fontsize = 15)
plt.show()

In [ ]:
#White-Test
pooled_OLS_dataset = pd.concat([dfreg, residuals_pooled_OLS], axis=1)
exog = sm.tools.tools.add_constant(dfreg[exog_vars]).fillna(0)
white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

#Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

Autocorrelation is positive and strong.

In [ ]:
#Non-Autocorrelation
# Durbin-Watson-Test
durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)

The regressions outcome shows a lack of significance for all the coefficients in both specifications so we cannot really prove that there exists a relationship between GDPpc and water withdrawals in this case. The Hausman test is thus not performed because it works to decide which model is best when both yield representative results, and this is not the case.

In [ ]:
# FE und RE model
exog_vars = ['log-gdppc','sq_log-gdppc']
exog = sm.add_constant(dfreg[exog_vars])
endog = dfreg['log-wtrwthpc']
# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
#print results
print(re_res)
print(fe_res)

In [ ]:
#Hausman test
def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov
    v_B = re.cov
    df = b[np.abs(b) < 1e8].size
    chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B)) 
 
    pval = stats.chi2.sf(chi2, df)
    return chi2, df, pval
hausman_results = hausman(fe_res, re_res) 
print('chi-Squared: '  + str(hausman_results[0]))
print('degrees of freedom: ' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

#### SECTOR ANALYSIS: AGRICULTURAL

In [ ]:
exog_vars = ['log-gdppc','sq_log-gdppc']
exog = sm.add_constant(dfreg[exog_vars])
endog = dfreg['log-wtrwthagpc']

In [ ]:
mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)

In [ ]:
# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

In [ ]:
#Homoskedasticity
#Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'green')
ax.axhline(0, color = 'b', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 10)
ax.set_ylabel('Residuals', fontsize = 10)
ax.set_title('Homoskedasticity Test', fontsize = 15)
plt.show()

For the case of the agricultural specification again we cannot discard heteroskedasticity in the error terms although we have the same situation as before, the White and the Breusch-Pagan tests contradict themselves.

In [ ]:
#White-Test
pooled_OLS_dataset = pd.concat([dfreg, residuals_pooled_OLS], axis=1)
exog = sm.tools.tools.add_constant(dfreg[exog_vars]).fillna(0)
white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

#Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

Again, autocorrelation is positive and even stronger than befroe, which leads us once again to preferring the panel specifications.

In [ ]:
#Non-Autocorrelation
# Durbin-Watson-Test
durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)

This time, all the coefficients are found to be significant in both specifications and while the linear term is positive the quadratic one is not, which is consistent with the EKC shape. 

In [ ]:
# FE und RE model
exog_vars = ['log-gdppc','sq_log-gdppc']
exog = sm.add_constant(dfreg[exog_vars])
endog = dfreg['log-wtrwthagpc']
# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
#print results
print(re_res)
print(fe_res)

The Hausman test does not support the use of fixed effects but, as we have already done numerous times, the preferred fixed specification is used. 

In [ ]:
def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov
    v_B = re.cov
    df = b[np.abs(b) < 1e8].size
    chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B)) 
 
    pval = stats.chi2.sf(chi2, df)
    return chi2, df, pval
hausman_results = hausman(fe_res, re_res) 
print('chi-Squared:'  + str(hausman_results[0]))
print('degrees of freedom:' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

##### CALCULATING THE TURNING POINT

We now can use these significant coefficients for the estimation of the turning point, that is calculated as follows: 

In [ ]:
fe_coeffs=fe_res.params
turn_point=np.exp(-fe_coeffs[1]/(2*fe_coeffs[2]))
print(turn_point)

This gives us a turning point of $11,121.72, which is consistent with other studies that have performed this kind of analysis.  

#### SECTOR ANALYSIS: DOMESTIC

In [ ]:
exog_vars = ['log-gdppc','sq_log-gdppc']
exog = sm.add_constant(dfreg[exog_vars])
endog = dfreg['log-wtrwthdompc']

In [ ]:
mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)

In [ ]:
# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

In [ ]:
#Homoskedasticity
#Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'green')
ax.axhline(0, color = 'b', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 10)
ax.set_ylabel('Residuals', fontsize = 10)
ax.set_title('Homoskedasticity Test', fontsize = 15)
plt.show()

The White and Breusch-Pagan show contradictory results again, but since the Breusch-Pagan p-value (<0.5) leads us to reject the null hypothesis we cannot say that there is no evidence of heteroskedasticity. 

In [ ]:
#White-Test
pooled_OLS_dataset = pd.concat([dfreg, residuals_pooled_OLS], axis=1)
exog = sm.tools.tools.add_constant(dfreg[exog_vars]).fillna(0)
white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

#Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

The autocorrelation is again positive.

In [ ]:
#Non-Autocorrelation
# Durbin-Watson-Test
durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)

In both the regressions only the constant term is significant at 10%, so we cannot conclude that it shows a relationship between water withdrawals and GDPpc. 

In [ ]:
# FE und RE model
exog_vars = ['log-gdppc','sq_log-gdppc']
exog = sm.add_constant(dfreg[exog_vars])
endog = dfreg['log-wtrwthdompc']
# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
#print results
print(re_res)
print(fe_res)

#### SECTOR ANALYSIS: INDUSTRIAL

In [ ]:
exog_vars = ['log-gdppc','sq_log-gdppc']
exog = sm.add_constant(dfreg[exog_vars])
endog = dfreg['log-wtrwthindpc']

In [ ]:
mod = PooledOLS(endog, exog)
pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)

In [ ]:
# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

In [ ]:
#Homoskedasticity
#Residuals-Plot for growing Variance Detection
fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'green')
ax.axhline(0, color = 'b', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 10)
ax.set_ylabel('Residuals', fontsize = 10)
ax.set_title('Homoskedasticity Test', fontsize = 15)
plt.show()

This time we can conclude that there is evidence of presence of heteroskedasticity in the residuals at a level of significance of 10% for the White test and 5% for the Breusch-Pagan test.

In [ ]:
#White-Test
pooled_OLS_dataset = pd.concat([dfreg, residuals_pooled_OLS], axis=1)
exog = sm.tools.tools.add_constant(dfreg[exog_vars]).fillna(0)
white_test_results = het_white(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

#Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooled_OLS_dataset['residual'], exog)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, breusch_pagan_test_results)))

Autocorrelation is positive and strong again.

In [ ]:
#Non-Autocorrelation
# Durbin-Watson-Test
durbin_watson_test_results = durbin_watson(pooled_OLS_dataset['residual']) 
print(durbin_watson_test_results)

The regressions output shows again a lack of significance in all the exogenous variables, so again we cannot state that there is a relationship between those and the endogenous one.

In [ ]:
# FE und RE model
exog_vars = ['log-gdppc','sq_log-gdppc']
exog = sm.add_constant(dfreg[exog_vars])
endog = dfreg['log-wtrwthindpc']
# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
#print results
print(re_res)
print(fe_res)

# Conclusion:
After performing the GLS Regressions we find that the only specification that gives out reasonable values while being significant and also being consistent is the Domestic Water withdrawals for the year 2012 (Sensitivity Analysis) which gives out a value of 10832 which is pretty close to the World Bank threshold for being considered a high income country (12600) dollars. But we have two major problems with the other regressions. The first one is that in the majority of the regressions give out significant values that respect the functional form of the EKC, the problem is that these values are so small that just by common sense shouldn't be taken too seriously. The second problem that we find is that the total water withdrawals for the year 2015 doesn't respect the functional form. We might think that these errors might be due to the structure of the databse or how is it written or maybe due to the presence of other problems like Multicollinearity. We also cannot reject the possibility that choosing another year might change the results and make everything normal. As an example the turning point difference between the 2017 period and 2012 in the case of Domestic Water Withdrawals. <br/>  In the fixed effects panel data is the agriculture withdrawals. we think that we are suffering from the same problems again. <br/> It might be interesting to study the effects of the EKC with other databases as we might see other values than the ones we got.